#Kaggle - San Francisco Crime classification

In [60]:
import graphlab

In [61]:
train = graphlab.SFrame.read_csv('train.csv')

PROGRESS: Finished parsing file /Users/vishnu/Dropbox/Private/study/big data/kaggle/sanfrancisco crime classification/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.800521 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str,str,str,str,str,str,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 360690 lines. Lines per second: 355504
PROGRESS: Finished parsing file /Users/vishnu/Dropbox/Private/study/big data/kaggle/sanfrancisco crime classification/train.csv
PROGRESS: Parsing completed. Parsed 878049 lines in 1.84616 secs.


#Explore the data

In [3]:
graphlab.canvas.set_target('ipynb')
train.show()

In [191]:
theft = train[train['Category']=='LARCENY/THEFT' or train['Category']=='VEHICLE THEFT']

In [67]:
train['HourOfDay']=train['Dates'].apply(lambda x: x.split(' ')[1].split(':')[0])
train['HourOfDay'].show()

In [68]:
train['Month'] = train['Dates'].apply(lambda x:x.split(' ')[0].split('-')[1])

In [171]:
def extractStreet(s):
    p = s.split(' ')
    l = len(p) - 1
    return str(p[l-1])

In [173]:
train['street']=train['Address'].apply(extractStreet)

In [174]:
train

Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution
2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED"
2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED"
2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED"
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKEDAUTO ...,Wednesday,NORTHERN,NONE
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKEDAUTO ...,Wednesday,PARK,NONE
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKEDAUTO ...,Wednesday,INGLESIDE,NONE
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKEDAUTO ...,Wednesday,RICHMOND,NONE
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKEDAUTO ...,Wednesday,CENTRAL,NONE


In [134]:
knn_model = graphlab.kmeans.create(train,50,features=['X','Y'])

PROGRESS: Initializing data.
PROGRESS: Batch size is larger than the input dataset. Switching to an exact Kmeans method.
PROGRESS: Choosing initial cluster centers with Kmeans++.
PROGRESS: +---------------+-----------+
PROGRESS: | Center number | Row index |
PROGRESS: +---------------+-----------+
PROGRESS: | 0             | 33812     |
PROGRESS: | 1             | 684406    |
PROGRESS: | 2             | 714798    |
PROGRESS: | 3             | 698486    |
PROGRESS: | 4             | 693889    |
PROGRESS: | 5             | 424569    |
PROGRESS: | 10            | 821466    |
PROGRESS: +---------------+-----------+
PROGRESS: Starting kmeans model training.
PROGRESS: Assigning points to initial cluster centers.
PROGRESS: +-----------+-------------------------------+
PROGRESS: | Iteration | Number of changed assignments |
PROGRESS: +-----------+-------------------------------+
PROGRESS: | 1         | 79229                         |
PROGRESS: | 2         | 45100                         |
PROG

In [195]:
knn_model['cluster_info']

X,Y,cluster_id,size,sum_squared_distance
-122.406572084,37.7868716073,0,51017,0.280413945093
-120.5,90.0,1,67,0.0
-122.447951176,37.7353138584,2,5615,0.272298621453
-122.500358248,37.7492223013,3,4399,0.187540132322
-122.419849366,37.7895503125,4,35956,0.383374516187
-122.43620101,37.7797105134,5,20326,0.19429771931
-122.386680065,37.7337186813,6,35597,0.970026371695
-122.408422613,37.7555500162,7,19602,0.256100593144
-122.413084451,37.7369438121,8,10389,0.319513269981
-122.44434967,37.7559568998,9,5316,0.167335841051


In [135]:
train['cluster']=knn_model['cluster_id']['cluster_id']

In [175]:
crimes = train['Category','cluster','street']

In [176]:
train_set,test_set = crimes.random_split(0.8,seed=0)

In [177]:
train_set.show()

In [178]:
lr_model = graphlab.logistic_classifier.create(train_set,
                                              target='Category',
                                              features=['cluster','street'],
                                              validation_set=test_set) 

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 702730
PROGRESS: Number of classes           : 39
PROGRESS: Number of feature columns   : 2
PROGRESS: Number of unpacked features : 2
PROGRESS: Number of coefficients    : 73036
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 4        | 0.000007  | 1.975636     | 0.218580          | 0.217284            |
PROGRESS: | 2         | 6        | 1.000000  | 11.515321    | 0.220363          | 0.216200            |
PROGRESS: | 3         | 7        | 1.000000  | 21.401726    | 0.235009 

In [179]:
lr_model

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 73036
Number of examples            : 702730
Number of classes             : 39
Number of feature columns     : 2
Number of unpacked features   : 2

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : auto
Solver iterations             : 10
Solver status                 : TERMINATED: Iteration limit reached.
Training time (sec)           : 103.3489

Settings
--------
Log-likelihood                : 1768188.9274

Highest Positive Coefficients
-----------------------------
street[DEWOLF]                : 12.1485
street[BURNS]                 : 11.8873
street[SOUTHARD]              : 11.591
street[AV]                    : 11.4636
street[WINTER]                : 11.3843

Lowest Negative Coefficients
----------------------------
street[MORE]                  : -3.1877
street[B

In [109]:
lr_model = graphlab.logistic_classifier.create(train_set,
                                              target='Category',
                                              features=['cluster'],
                                              validation_set=test_set) 

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 702730
PROGRESS: Number of classes           : 39
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 1
PROGRESS: Number of coefficients    : 76
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 10.289724    | 0.198951          | 0.200155            |
PROGRESS: | 2         | 3        | 16.950637    | 0.028408          | 0.027978            |
PROGRESS: | 3         | 4        | 23.937320    | 0.107504          | 0.108100            |
PROGRESS: +-----------+----------+-

In [127]:
lr_model

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 76
Number of examples            : 702730
Number of classes             : 39
Number of feature columns     : 1
Number of unpacked features   : 1

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : auto
Solver iterations             : 3
Solver status                 : TERMINATED: Terminated due to numerical overflow error. 
Step-size parameter may be too large or data may contain NaNs.
Training time (sec)           : 23.9381

Settings
--------
Log-likelihood                : inf

Highest Positive Coefficients
-----------------------------
No Positive Coefficients      : 

Lowest Negative Coefficients
----------------------------
(intercept)                   : -35326169.5222
(intercept)                   : -35326169.1608
(intercept)                   : -35326168.5117
(int